In [29]:
nums = [int(x) for x in open("../data/01.txt").readlines()]

def sliding_window(arr, n):
    for i in range(0, len(arr) - n + 1):
        yield arr[i:i+n]

res1 = sum(x[1] > x[0] for x in sliding_window(nums, 2))
res2 = sum(x[1] > x[0] for x in sliding_window(
    [sum(y) for y in sliding_window(nums, 3)], 
    2))

print(f'Answer 1: {res1}\nAnswer 2: {res2}')

Answer 1: 1557
Answer 2: 1608


In [51]:
from functools import reduce

commands = [(x.split()[0], int(x.split()[1])) 
             for x in open("../data/02.txt").readlines()]

def step1(pos, command):
    (x, depth) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth - offs)
    elif cmd == 'down':
        return (x, depth + offs)
    elif cmd == 'forward':
        return (x + offs, depth)
    
def step2(pos, command):
    (x, depth, aim) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth, aim - offs)
    elif cmd == 'down':
        return (x, depth, aim + offs)
    elif cmd == 'forward':
        return (x + offs, depth + aim * offs, aim)
        
pos1 = reduce(step1, commands, (0, 0))
pos2 = reduce(step2, commands, (0, 0, 0))
print(f'Answer 1: {pos1[0] * pos1[1]}\nAnswer 2: {pos2[0] * pos2[1]}')

Answer 1: 1728414
Answer 2: 1765720035
